"# Introduction to the Monte Carlo method

----

Start by defining the [Gibbs (or Boltzmann) distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution):
$$P(\alpha) = e^{-E(\alpha)/kT}$$
this expression, defines the probability of observing a particular configuration of spins, $\alpha$. 
As you can see, the probability of $\alpha$ decays exponentially with increasing energy of $\alpha$, $E(\alpha)$,
where $k$ is the Boltzmann constant, $k = 1.38064852 \times 10^{-23} J/K$
and $T$ is the temperature in Kelvin. 

## What defines the energy of a configuration of spins? 
Given a configuration of spins (e.g., $\uparrow\downarrow\downarrow\uparrow\downarrow$) we can define the energy using what is referred to as an Ising Hamiltonian:
$$ \hat{H}' = \frac{\hat{H}}{k} = -\frac{J}{k}\sum_{<ij>} s_is_j,$$
where, $s_i=1$ if the $i^{th}$ spin is `up` and $s_i=-1$ if it is `down`, and the brackets $<ij>$ indicate a sum over spins that are connected,
and $J$ is a constant that determines the energy scale. 
The energy here has been divided by the Boltzmann constant to yield units of temperature. 
Let's consider the following case, which has the sites connected in a single 1D line:
$$\alpha = \uparrow-\downarrow-\downarrow-\uparrow-\downarrow.$$ 
What is the energy of such a configuration?
$$ E(\alpha)' = J/k(-1 + 1 - 1 - 1) = \frac{E(\alpha)}{k} = -2J/k$$

## P1: Write a class that defines a spin configuration

In [134]:
#def spin configuration class

class SpinConfig():
    
    def __init__(self):
       print("spinconfig created")
    
    def input_decimal(self, decimal_Input,site_number):
        self.decimal_Input=decimal_Input
        self.site_number = site_number
        spinlist=[]
            
        for element in bin(self.decimal_Input)[2:]:
            spinlist.append(int(element))
            
        while len(spinlist) < self.site_number:
            spinlist = [0]+spinlist
            
        return spinlist

    
    def input_p_m(self, p_m_Input):
        self.p_m_Input = p_m_Input
        spinlist2=list()
        for element in self.p_m_Input:
            if element =="+":
                spinlist2.append(1)
            elif element == "-":
                spinlist2.append(0)
            else:
                pass
        return spinlist2



In [111]:
mySpin = SpinConfig()

spinconfig created


In [112]:
mySpin.input_decimal(3,6)

[0, 0, 0, 0, 1, 1]

In [113]:
mySpin.input_p_m("+++")

[1, 1, 1]

## P2: Write a class that defines the 1D hamiltonian, containing a function that computes the energy of a configuration

In [135]:
#coupling Hamiltonian class def:
class Hamiltonian():
    
    def __init__(self,J=-2,k=1.38064852* 10 **(-23),u=1.1): 
        
        print("class Hamiltnoian created")
        
        self.k = k
        self.u = u
        self.J = J


    def energy(self, spinconfig):

        self.spinconfig = spinconfig
        energy=0
        """
        Energy from the external field:
        H_external = Sum over i of u * spin[i]
        """
        for eachspin in self.spinconfig:
            if eachspin == 1:
                energy+= self.u * 1
            elif eachspin == 0:
                energy += self.u * (-1)
            else:
                print("Spin input error")
                

        """
        Energy from coupling the nearest neighbor spin:
        H_c = -J/k * spin[i] * spin[i+1]
        """
        newList = self.spinconfig[1:]
        newList.append(self.spinconfig[0])
        for spinx, spiny in zip(self.spinconfig, newList):
            if spinx==spiny:
                energy += -self.J/self.k * 1
            elif spinx!=spiny:
                energy += -self.J/self.k * (-1)
            else:
                print("Type error spininput")
                
        return energy


In [115]:
myH = Hamiltonian()

class Hamiltnoian created


In [116]:
myH.energy([0,1,0,1,1])

-4.345783820490389e+23

## Q3: What is the energy for (++-+---+--+)?

In [117]:
mySpin = SpinConfig()

spinconfig created


In [118]:
mySpin.input_p_m("++-+---+--+")

[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1]

In [119]:
myH.energy(mySpin.input_p_m("++-+---+--+"))

-1.4485946068301296e+23

## Q2: How many configurations are possible for:

(a) N=10?

In [96]:
2**10

1024

(b) N=100?

In [97]:
2**100

1267650600228229401496703205376

(c) N=1000?

In [98]:
2**1000

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376

## Properties
For any fixed state, $\alpha$, the `magnetization` ($M$) is proportional to the _excess_ number of spins pointing up or down while the energy is given by the
Hamiltonian:
$$M(\alpha) = N_{\text{up}}(\alpha) - N_{\text{down}}(\alpha).$$
As a dynamical, fluctuating system, each time you measure the magnetization, the system might be in a different state ($\alpha$) and so you'll get a different number!
However, we already know what the probability of measuring any particular $\alpha$ is, so in order to compute the average magnetization, $\left<M\right>$, we just need to multiply the magnetization of each possible configuration times the probability of it being measured, and then add them all up!
$$ \left<M\right> = \sum_\alpha M(\alpha)P(\alpha).$$
In fact, any average value can be obtained by adding up the value of an individual configuration multiplied by it's probability:
$$ \left<E\right> = \sum_\alpha E(\alpha)P(\alpha).$$

This means that to obtain any average value (also known as an `expectation value`) computationally, we must compute the both the value and probability of all possible configurations. This becomes extremely expensive as the number of spins ($N$) increases. 

In [179]:
import math

k = 1.38064852*10**-23
mySpin = SpinConfig()
myH = Hamiltonian()

#for myN=8 spin number, total possible configuration number: 
myN=8
iMax = 2**myN

spinconfig created
class Hamiltnoian created


1. Averaged E:  <E>

In [154]:
spinList=[]
i=0
while i<iMax:
    spinList.append(mySpin.input_decimal(i,myN))
    i+=1

In [182]:
energyList=[]
i=0
while i<iMax:
    energyList.append(myH.energy(spinList[i]))
    i+=1

In [184]:
energyList[2]

5.794378427320518e+23

In [191]:
# partition function myZ:
def myZ(T):
    
    Zval = 0
    i=0
    while i<iMax:
        Zval += math.exp(-energyList[i]/(k*T))
        i+=1
        
    return Zval

In [192]:
myZ(10)

OverflowError: math range error

## P3: Write a function that computes the magnetization of a spin configuration

2.718281828459045